In [1]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /Users/sambarati/Documents/GitHub/mlfs-book
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [16]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [17]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2025-11-10 19:01:46,002 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-10 19:01:46,008 INFO: Initializing external client
2025-11-10 19:01:46,008 INFO: Base URL: https://c.app.hopsworks.ai:443
Connection closed.
2025-11-10 19:01:46,008 INFO: Initializing external client
2025-11-10 19:01:46,008 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-10 19:01:47,675 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1267871

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1267871


'{"country": "France", "city": "Paris", "street": "Boulevard Peripherique Est", "aqicn_url": "https://api.waqi.info/feed/@3088/", "latitude": 48.86, "longitude": 2.32}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [18]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=2,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [19]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,26.0,France,Paris,Boulevard Peripherique Est,2025-11-10,https://api.waqi.info/feed/@3088/


In [20]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 176.0+ bytes


## <span style='color:#ff5f27'> 🔄 Calculate Lagged Features</span>

To create the lagged features (pm_25_1_day_lag, pm_25_2_day_lag, pm_25_3_day_lag), we need to fetch the last 3 days of PM2.5 values from the feature group.

In [ ]:
# Fetch the last 3 days of PM2.5 data to calculate lagged features
three_days_ago = today - datetime.timedelta(days=3)

# Read historical data from the feature group (version 2) - FILTER BY CITY AND STREET
historical_aq_df = air_quality_fg.filter(
    (air_quality_fg.date >= three_days_ago) & 
    (air_quality_fg.city == city) & 
    (air_quality_fg.street == street)
).read()

# Sort by date to ensure proper ordering
historical_aq_df = historical_aq_df.sort_values(by='date')

# Get the last 3 PM2.5 values (most recent first)
if len(historical_aq_df) >= 3:
    pm25_values = historical_aq_df['pm25'].tail(3).values
    aq_today_df['pm_25_1_day_lag'] = pm25_values[-1]  # Yesterday
    aq_today_df['pm_25_2_day_lag'] = pm25_values[-2]  # 2 days ago
    aq_today_df['pm_25_3_day_lag'] = pm25_values[-3]  # 3 days ago
else:
    print(f"Warning: Not enough historical data. Found {len(historical_aq_df)} days, need 3.")
    # Set to None or skip insertion if not enough data
    aq_today_df['pm_25_1_day_lag'] = None
    aq_today_df['pm_25_2_day_lag'] = None
    aq_today_df['pm_25_3_day_lag'] = None

print(f"Lagged features calculated:")
print(f"  1-day lag (yesterday): {aq_today_df['pm_25_1_day_lag'].values[0]}")
print(f"  2-day lag: {aq_today_df['pm_25_2_day_lag'].values[0]}")
print(f"  3-day lag: {aq_today_df['pm_25_3_day_lag'].values[0]}")

aq_today_df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.08s) 
Lagged features calculated:
  1-day lag (yesterday): None
  2-day lag: None
  3-day lag: None


,pm25,country,city,street,date,url,pm_25_1_day_lag,pm_25_2_day_lag,pm_25_3_day_lag
0,26.0,France,Paris,Boulevard Peripherique Est,2025-11-10,https://api.waqi.info/feed/@3088/,None,None,None


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [8]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 48.75°N 2.25°E
Elevation 46.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-10,11.55,0.0,14.618837,170.073822,Paris
1,2025-11-11,12.30,0.0,16.235960,176.186005,Paris
2,2025-11-12,15.15,0.0,11.620809,163.810730,Paris
3,2025-11-13,18.75,0.0,9.028754,184.573837,Paris
4,2025-11-14,15.95,0.0,4.582052,135.000107,Paris
5,2025-11-15,12.60,0.0,8.891343,31.759439,Paris
6,2025-11-16,5.90,0.0,10.703569,47.726372,Paris


In [9]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 356.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [10]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-10 18:30:28,467 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1267871/fs/1262659/fg/1638064
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1267871/fs/1262659/fg/1638064


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00



Launching job: air_quality_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1267871/jobs/named/air_quality_2_offline_fg_materialization/executions
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1267871/jobs/named/air_quality_2_offline_fg_materialization/executions


(Job('air_quality_2_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 735348
         }
       },
       "result": {
         "observed_value": 37.999996185302734,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-10T05:30:28.000467Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "succe

In [11]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-10 18:30:46,208 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1267871/fs/1262659/fg/1595979
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1267871/fs/1262659/fg/1595979


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time: 00:00



Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1267871/jobs/named/weather_1_offline_fg_materialization/executions
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1267871/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-10 18:31:04,560 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-10 18:31:04,560 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-10 18:31:07,755 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-10 18:31:07,755 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-10 18:33:28,148 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-10 18:33:28,148 INFO: Waiting for execution to finish. C

(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 733221
         }
       },
       "result": {
         "observed_value": 4.5820512771606445,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-10T05:30:46.000207Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type":

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
